In [56]:
import requests
import pandas as pd
import json
import time

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

Pulling Data From API

In [ ]:
url = "https://zillow56.p.rapidapi.com/search"

headers = {
	"X-RapidAPI-Key": "aede897c5cmshc2c71b7d354abd2p1382e3jsn59e7e1a1fd45",
	"X-RapidAPI-Host": "zillow56.p.rapidapi.com"
}

lst_1 = []

for page in range(1, 21):
    querystring = {"location":"Utah County, UT", 
                "sortSelection":"days", 
                "page":f"{page}"}

    response = requests.get(url, headers=headers, params=querystring)

    data = response.json()
    df__ = pd.json_normalize(data['results'])
    lst_1.append(df__)
    time.sleep(4)

df = pd.concat(lst_1)


In [ ]:
# df.to_csv('zillow_orig.csv')

Cleaning the Data

In [61]:
## Because I have limited API requests ##

df_orig = pd.read_csv('zillow_orig.csv')


In [62]:
print(df_orig.columns)

Index(['Unnamed: 0', 'bathrooms', 'bedrooms', 'city', 'country', 'currency',
       'daysOnZillow', 'group_type', 'homeStatus', 'homeStatusForHDP',
       'homeType', 'imgSrc', 'isFeatured', 'isNonOwnerOccupied',
       'isPreforeclosureAuction', 'isPremierBuilder', 'isShowcaseListing',
       'isUnmappable', 'isZillowOwned', 'latitude', 'livingArea', 'longitude',
       'newConstructionType', 'price', 'priceForHDP', 'priceSuffix',
       'providerListingID', 'rentZestimate', 'shouldHighlight', 'state',
       'streetAddress', 'unit', 'zipcode', 'zpid',
       'listing_sub_type.is_newHome', 'lotAreaUnit', 'lotAreaValue',
       'zestimate', 'taxAssessedValue', 'listing_sub_type.is_FSBA',
       'openHouse', 'listing_sub_type.is_openHouse',
       'open_house_info.open_house_showing', 'datePriceChanged', 'priceChange',
       'priceReduction'],
      dtype='object')


In [67]:
# Dropping columns of no interest
df = df_orig[['price', 'bathrooms', 'bedrooms', 'city', 'homeType', 'livingArea', 'zipcode', 'priceReduction', 'daysOnZillow']]
df = df[df['homeType'] != 'LOT']
df

,price,bathrooms,bedrooms,city,homeType,livingArea,zipcode,priceReduction,daysOnZillow
0,414990.0,3.0,3.0,American Fork,TOWNHOUSE,1619.0,84003,NaN,-1
1,395990.0,3.0,2.0,American Fork,TOWNHOUSE,1421.0,84003,NaN,-1
2,461990.0,3.0,4.0,American Fork,TOWNHOUSE,2078.0,84003,NaN,-1
3,413990.0,3.0,3.0,American Fork,TOWNHOUSE,1645.0,84003,NaN,-1
4,413990.0,3.0,3.0,American Fork,TOWNHOUSE,1591.0,84003,NaN,-1
5,439990.0,3.0,3.0,American Fork,TOWNHOUSE,1914.0,84003,NaN,-1
6,429990.0,3.0,2.0,American Fork,TOWNHOUSE,1768.0,84003,NaN,-1
7,455990.0,3.0,3.0,American Fork,TOWNHOUSE,2120.0,84003,NaN,-1
8,453990.0,3.0,3.0,American Fork,TOWNHOUSE,2086.0,84003,NaN,-1
9,898000.0,5.0,4.0,Spanish Fork,SINGLE_FAMILY,4900.0,84660,NaN,0
